# Notebook to generate indicators module

In [99]:
from pprint import pformat

from mintalib import core

from inspect import Signature

from mintalib.samples import sample_prices

from __project__ import PKGDIR

In [100]:
prices = sample_prices()
prices.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11033 entries, 1980-12-12 to 2024-09-18
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    11033 non-null  float64
 1   high    11033 non-null  float64
 2   low     11033 non-null  float64
 3   close   11033 non-null  float64
 4   volume  11033 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 517.2 KB


In [101]:
import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [102]:
def core_functions():
    return list(k for k, v in vars(core).items() if k.isupper() and callable(v))

core_functions()

['AVGPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'MIDPRICE',
 'PRICE',
 'CROSSOVER',
 'CROSSUNDER',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'LOG',
 'EXP',
 'ROC',
 'DIFF',
 'MIN',
 'MAX',
 'SUM',
 'MAD',
 'STDEV',
 'SMA',
 'EMA',
 'RMA',
 'WMA',
 'HMA',
 'DEMA',
 'TEMA',
 'MA',
 'RSI',
 'PLUSDI',
 'MINUSDI',
 'ADX',
 'TRANGE',
 'ATR',
 'NATR',
 'LATR',
 'SAR',
 'CCI',
 'CMF',
 'MFI',
 'BOP',
 'BBANDS',
 'KELTNER',
 'KER',
 'KAMA',
 'MACD',
 'PPO',
 'SLOPE',
 'RVALUE',
 'FORECAST',
 'STOCH',
 'STREAK',
 'EVAL']

In [103]:
# Sample code for indicator

from mintalib.core import wrap_indicator
from mintalib.model import FuncIndicator

@wrap_indicator(core.SMA)
def SMA(period: int = 3):
    return FuncIndicator.wrap(core.SMA, params=locals())


In [104]:
PREAMBLE = '''
""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


'''


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.annotation == "int":
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(param.default) in (int, float, bool):
            return param.replace(annotation=type(param.default))

        return param


    newparams = parameters[1:]
    newparams = [annotate(p) for p in newparams]

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_indicator({qname})\n"
    code += f"def {name}{newsig}:\n"
    code += f"    return FuncIndicator({qname}, params=locals())\n"


    return code


print(make_indicator(core.EMA))

@wrap_indicator(core.EMA)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    return FuncIndicator(core.EMA, params=locals())



In [105]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    print(code)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


@wrap_indicator(core.EMA)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    return FuncIndicator(core.EMA, params=locals())



<module 'mintalib.functions'>

In [106]:

def make_indicators(names=None):
    if names is None:
        names = core_functions()

    output = PREAMBLE

    buffer = pformat(names, width=75, compact=True, indent=4)
    buffer = buffer.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{buffer}\n]\n"

    output += "\n\n"

    for name in names:
        func = getattr(core, name)
        text = make_indicator(func)
        output += text + "\n\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"

    return output


code = make_indicators()
print(code[:1024], "...")


""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


__all__ = [
    'AVGPRICE', 'TYPPRICE', 'WCLPRICE', 'MIDPRICE', 'PRICE', 'CROSSOVER',
    'CROSSUNDER', 'FLAG_ABOVE', 'FLAG_BELOW', 'FLAG_INVERT', 'FLAG_UPDOWN',
    'LOG', 'EXP', 'ROC', 'DIFF', 'MIN', 'MAX', 'SUM', 'MAD', 'STDEV',
    'SMA', 'EMA', 'RMA', 'WMA', 'HMA', 'DEMA', 'TEMA', 'MA', 'RSI',
    'PLUSDI', 'MINUSDI', 'ADX', 'TRANGE', 'ATR', 'NATR', 'LATR', 'SAR',
    'CCI', 'CMF', 'MFI', 'BOP', 'BBANDS', 'KELTNER', 'KER', 'KAMA', 'MACD',
    'PPO', 'SLOPE', 'RVALUE', 'FORECAST', 'STOCH', 'STREAK', 'EVAL'
]


@wrap_indicator(core.AVGPRICE)
def AVGPRICE():
    return FuncIndicator(core.AVGPRICE, params=locals())


@wrap_indicator(core.TYPPRICE)
def TYPPRICE():
    return FuncIndicator(core.TYPPRICE, params=locals())


@wrap_indicator(core.WCLPRICE)
def WCLPRICE():
    return FuncIndi

In [107]:
code = make_indicators()

indicators = new_module("mintalib.indicators", code)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'DEMA',
 'DIFF',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'FORECAST',
 'FuncIndicator',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'nan',
 'wrap_indicator']

In [108]:
assert callable(indicators.SMA)
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0


In [109]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


7497

In [110]:
from mintalib import indicators

In [111]:
help(indicators)

Help on module mintalib.indicators in mintalib:

NAME
    mintalib.indicators - Mintalib Indicators

FUNCTIONS
    ADX(period: int = 14)
        Average Directional Index
    
    ATR(period: int = 14)
        Average True Range
    
    AVGPRICE()
        Average Price
    
    BBANDS(period: int = 20, nbdev: float = 2.0)
        Bollinger Bands
    
    BOP(period: int = 20)
        Balance of Power
    
    CCI(period: int = 20)
        Commodity Channel Index
    
    CMF(period: int = 20)
        Chaikin Money Flow
    
    CROSSOVER(level: float = 0.0, *, item: str = None)
        Cross Over
    
    CROSSUNDER(level: float = 0.0, *, item: str = None)
        Cross Under
    
    DEMA(period: int, *, item: str = None)
        Double Exponential Moving Average
    
    DIFF(period: int = 1, *, item: str = None)
        Difference
    
    EMA(period: int, *, adjust: bool = False, item: str = None)
        Exponential Moving Average
        
        Args:
            period (int) :

In [112]:
assert callable(indicators.SMA)
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0